In [10]:
import numpy as np
import pandas as pd
import os

In [24]:
dataLocation="../Data/KaggleData/US_Funds_dataset_from_Yahoo_Finance/ETF prices.csv"
rawData=pd.read_csv(dataLocation).drop(columns=["adj_close","price_date"])
rawData.info()
# turn float64 to float 32

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3866030 entries, 0 to 3866029
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fund_symbol  object 
 1   open         float64
 2   high         float64
 3   low          float64
 4   close        float64
 5   volume       int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 177.0+ MB


In [42]:
# stocks=rawData["fund_symbol"].unique()
# it takes abt 5 mins to do this find another way

# dfs= [0]* stocks.shape[0]
# for i,stock in enumerate(stocks):
#     print(i)
#     dfs[i]=rawData[rawData["fund_symbol"] == stock]
# rawData=None

In [4]:
lookback_window=10 #I don't have any idea which number is best
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


num_features=5 #for OHLC and volume
model = Sequential()
model.add(LSTM(32), return_sequences=True, input_shape=(lookback_window, num_features)))
model.add(Dense(1))  # Predicting one value, like 'close'
model.compile(optimizer='adam', loss='mean_squared_error')